# 第3日の課題-2

scikit-learnのRandomizedSearchCVを用いて、bostonデータに対するrbfカーネルSVR回帰の最適なパラメータを求めよ。

In [0]:
import scipy as sp
import numpy as np
from sklearn.datasets import load_boston
from sklearn.svm import SVR
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV

bostonデータを読み込み、パターン行列Xと教師ベクトルyにデータを格納します。

In [0]:
boston = load_boston()
X = boston.data
y = boston.target

### scipy.stats.exponの動作確認

[scipy.stats.expon](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.expon.html)は確率密度関数 $\frac{exp(-x)}{scale}$ for $x \geq 0$を表します。rvsメソッドはその関数に基づいて乱数を発生し、ndarrayに格納します。適当なscaleで乱数を10個発生させてみます。

In [3]:
sp.stats.expon.rvs(size=10)

array([0.5490345 , 0.36241097, 0.76609452, 0.23688289, 0.51819958,
       0.05952214, 0.21772259, 0.18627408, 0.65875259, 3.02708066])

## 回帰問題への適用

bostonデータに対する[SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)回帰でRandomized searchを行います。

回帰器のインスタンスの作成

In [4]:
svr = SVR()
svr

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

スラック変数の重みCと、RBFカーネルの係数gammaの値を乱数で生成する確率分布をparamsとします。

In [0]:
params = {'C': sp.stats.expon(scale=10), 'gamma': sp.stats.expon(scale=0.1)}

回帰の場合は、ShuffleSplitのインスタンスを作成し、それをRandomizedSearchCVのcvパラメータの値として与えます。

In [6]:
cv = ShuffleSplit(n_splits=3)
reg = RandomizedSearchCV(svr, params, cv=cv, scoring='r2', n_iter=100, return_train_score=True)
reg.fit(X,y) 

RandomizedSearchCV(cv=ShuffleSplit(n_splits=3, random_state=None, test_size=None, train_size=None),
                   error_score='raise-deprecating',
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, shrinking=True,
                                 tol=0.001, verbose=False),
                   iid='warn', n_iter=100, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f720f9ce978>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f720f9ceb00>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring='r2', verbose=0)

結果の詳細表示で、交差確認がうまく適用できていることを確認します。

In [7]:
reg.cv_results_

{'mean_fit_time': array([0.0166409 , 0.01585754, 0.01934417, 0.01789705, 0.03363196,
        0.01883737, 0.01772666, 0.03374847, 0.02265573, 0.02130739,
        0.0291632 , 0.02210522, 0.02631418, 0.03182626, 0.01835759,
        0.02264563, 0.01652384, 0.03581254, 0.03447096, 0.03237549,
        0.03974136, 0.02621325, 0.01753092, 0.0226558 , 0.02387492,
        0.03362298, 0.01603874, 0.01906252, 0.02880645, 0.03590926,
        0.02872348, 0.02653511, 0.02874613, 0.0270586 , 0.04028209,
        0.02878388, 0.02483813, 0.03091311, 0.03272764, 0.02492857,
        0.02484226, 0.0323592 , 0.02869821, 0.02550443, 0.03574491,
        0.02436813, 0.03428706, 0.02605494, 0.02126932, 0.02713903,
        0.02319129, 0.02309108, 0.03572424, 0.01485189, 0.02592381,
        0.02489146, 0.03391035, 0.0340511 , 0.02696872, 0.04089228,
        0.03361146, 0.02668389, 0.01529257, 0.02792692, 0.02779118,
        0.03031588, 0.02148239, 0.0317843 , 0.02148644, 0.02555005,
        0.01791398, 0.03195477,

結果のまとめ（全結果の表示、最適なパラメータ・スコア）

In [8]:
re = reg.cv_results_
for params, mean_score, std_score in zip(re['params'], re['mean_test_score'], re['std_test_score']):
    print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, std_score, params))

0.010 (+/- 0.003) for {'C': 2.2205138559567144, 'gamma': 0.2407861578384948}
0.008 (+/- 0.003) for {'C': 0.5401677681801562, 'gamma': 0.1015522172968382}
0.191 (+/- 0.048) for {'C': 1.5578700262188585, 'gamma': 0.017671264287614976}
0.030 (+/- 0.007) for {'C': 3.596343897980697, 'gamma': 0.19913337512764723}
0.440 (+/- 0.165) for {'C': 35.23827892916151, 'gamma': 0.025497710173880767}
0.172 (+/- 0.049) for {'C': 0.9396369004918749, 'gamma': 0.012779294743369848}
0.079 (+/- 0.022) for {'C': 1.4105707019765037, 'gamma': 0.049978062688752614}
0.282 (+/- 0.119) for {'C': 40.39570979636187, 'gamma': 0.07406750228933186}
0.143 (+/- 0.034) for {'C': 3.0784914549155817, 'gamma': 0.048761578800676025}
0.068 (+/- 0.016) for {'C': 3.5212738719887904, 'gamma': 0.11611919214347668}
0.115 (+/- 0.036) for {'C': 19.6567572138564, 'gamma': 0.1937751507355121}
0.283 (+/- 0.067) for {'C': 2.7504459781302515, 'gamma': 0.014665266798614905}
0.197 (+/- 0.055) for {'C': 8.015622820172329, 'gamma': 0.07080665

In [9]:
reg.best_params_

{'C': 15.759236521159327, 'gamma': 0.0009081468421428314}

In [10]:
reg.best_score_

0.6038485061110216